In [ ]:
import numpy as np

def loadFLIMbin(binpath=None, buffer=None, num_bins=3):
    ''' Reads decay matrix from a FLIM .bin file.
        Notes: as of right now does not fully support all decay matrix dimensions.
        Takes a path to a .bin file
        Only works if the 3rd dimension mod binning is equal to 2
        Return a FLIMtuple (decaymatrix, time_res_bin, img_res, time_interval) '''
    dtype = np.int32
    if binpath is not None:
        data = np.fromfile(binpath, dtype)
    elif buffer is not None:
        data = np.frombuffer(buffer, dtype)
    else:
        raise Exception('Has to specify either a binpath or a buffer')
    time_res = data[3]
    img_res = data[0]
    decaymatrix_raw = data[5:].reshape((time_res, img_res, img_res), order='F')
    decaymatrix_raw = decaymatrix_raw.transpose((2,1,0))

    # binning
    if num_bins > 1:
        shape = decaymatrix_raw.shape
        shape = (*shape[:-1], num_bins, int((shape[-1]-2)/num_bins))
        n_time_points = shape[-2]*shape[-1]
        decaymatrix = decaymatrix_raw[:,:,time_res-n_time_points:time_res]
        decaymatrix = decaymatrix.reshape(shape, order='F')
        decaymatrix = np.mean(decaymatrix, axis=2)
    else:
        decaymatrix = decaymatrix_raw

    time_period_ns = 12.508416*num_bins/time_res;
    time_res_bin = time_res/num_bins;

    return (decaymatrix, time_res_bin, img_res, time_period_ns)